In [1]:
!pip install boto3

In [2]:
!pip install s3fs

In [1]:
from scipy import io as sio
import numpy as np
import csv
import matplotlib as mp
import matplotlib.pyplot as plt
from numpy.lib import recfunctions
import datetime
import numpy.lib.recfunctions as recfn
import pandas as pd
import glob
import json
import s3fs
import io
import gzip
import csv

In [2]:
# Function to read S3 keys from bucket
import boto3
s3 = boto3.resource('s3')
def s3_bucket_object_keys(bucket_name= 'iia-vault-telemetry-practice-unzipped'):
    bucket = s3.Bucket(bucket_name)
    key_list=[]
    for key in bucket.objects.all():
        key_list.append(key)
    return(key_list)

In [3]:
# Function to return mat files
from io import BytesIO
def load_mat_file(s3_key, bucketname= 'iia-vault-telemetry-practice-unzipped'):
    s3 = boto3.resource('s3')
    obj = s3.Object(bucketname, s3_key)
    inMATFile = obj.get()['Body'].read()
    raw_mat = sio.loadmat(BytesIO(inMATFile))
    return(raw_mat)

In [4]:
# Retrived Key list
start = datetime.datetime.now()
key_list = s3_bucket_object_keys()
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:18.080250


In [5]:
# Extract the names of the features (i.e. the keys in the raw python dictionary)
# starting point: 189 features
# Delete 3 specific keys since they are metadata and not associated with the blackbox data
# intermediate total: 186
# Delete all features that have resolution < 1Hz since these are  static and non-relevant
# Final total: 163

def extract_features(raw_data):
    
    features = raw_data.keys()
    features_25 = []
    features_1 = []
    data_def=[]
    number_of_features =  len(features) 
    #print number_of_features
#     del features[137]
#     del features[160]
#     del features[181]
    for key in features:
        if key == '__header__' or key == '__version__' or key == '__globals__':
            continue
        data =  raw_data[key]['data'][0][0].T
        rate =  raw_data[key]['Rate'][0][0][0][0]
        try:
            units= raw_data[key]['Units'][0][0][0]
        except:
            units= None
        dtype = raw_data[key]['data'][0][0][0][0].dtype
        desc = raw_data[key][0][0][3][0]
        data_def.append([key, len(data),rate, units, dtype, desc, data])
#         features_scrubbed.append(key)
        if rate == 0.25: # sample rate < 1Hz
            #interpolate_to_1hz(data)
            features_25.append(key)
        else: # sample rate > 1Hz
            features_1.append(key)
#     #print len(features_scrubbed)
    return features_25, features_1, data_def

# Create recarray with feature vectors. Normalize all feature data vectors to fit the 1Hz resolution
def feature_vector_array(raw_data, features_25, features_1,file_name):
    arr_dict = {}
    # Setup of final array
    # arr = np.zeros((len(raw_data['SAT']['data'][0][0])),dtype=[('timestamp', np.str_, 20)]) #'datetime64[us]'
    ff = features_1+features_25
    d_length = len(ff)
    t_length = len(raw_data['SAT']['data'][0][0])
    arrs_1d = [0]
    
    # selecting the base timestamp (first record in file) and filling first column of csv with timestamp data based on it
    year       = raw_data['DATE_YEAR']['data'][0][0][0,0]
    month      = raw_data['DATE_MONTH']['data'][0][0][0,0]
    day        = raw_data['DATE_DAY']['data'][0][0][0,0]
    hour       = raw_data['GMT_HOUR']['data'][0][0][0,0]
    minute     = raw_data['GMT_MINUTE']['data'][0][0][0,0]
    sec        = raw_data['GMT_SEC']['data'][0][0][0,0]

    # 687200403251247.mat
    if month <1 or month > 12:
      year = int(file_name[3:7])
      month = int(file_name[7:9])
      day = int(file_name[9:11])
      hour = int(file_name[11:13])
      minute = int(file_name[13:15])
      sec = 0
    

    base = datetime.datetime(year, month, day, hour, minute, sec)
    date_list = [base + datetime.timedelta(seconds=x) for x in range(0, t_length)]
    dates = np.array(date_list)
#     flghtnum= np.repeat(file_name, len(dates))
    # arr['timestamp'] = dates
    feature_dict['timestamp'] = (dates.T).tolist()
    
    # Filling all feature vector columns in the recarray
    for key in features_1:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = data[0:data.size:rate]
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = (norm_v.T).tolist()[0]
    size_1hz = len(norm_v)
    for key in features_25:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = np.resize(data,(size_1hz,1))
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = (norm_v.T).tolist()[0]
        # print((norm_v.T).tolist()[0])
    return

def feature_dictionary(feature_dict,raw_data,features_25, features_1,file_name):

    # Setup of final array
    # arr = np.zeros((len(raw_data['SAT']['data'][0][0])),dtype=[('timestamp', np.str_, 20)]) #'datetime64[us]'
    ff = features_1+features_25
    d_length = len(ff)
    t_length = len(raw_data['SAT']['data'][0][0])
    arrs_1d = [0]
    
    # selecting the base timestamp (first record in file) and filling first column of csv with timestamp data based on it
    year       = raw_data['DATE_YEAR']['data'][0][0][0,0]
    month      = raw_data['DATE_MONTH']['data'][0][0][0,0]
    day        = raw_data['DATE_DAY']['data'][0][0][0,0]
    hour       = raw_data['GMT_HOUR']['data'][0][0][0,0]
    minute     = raw_data['GMT_MINUTE']['data'][0][0][0,0]
    sec        = raw_data['GMT_SEC']['data'][0][0][0,0]

    # 687200403251247.mat
    if month <1 or month > 12:
      year = int(file_name[3:7])
      month = int(file_name[7:9])
      day = int(file_name[9:11])
      hour = int(file_name[11:13])
      minute = int(file_name[13:15])
      sec = 0
        
    if hour <0 or hour > 23:
      year = int(file_name[3:7])
      month = int(file_name[7:9])
      day = int(file_name[9:11])
      hour = int(file_name[11:13])
      minute = int(file_name[13:15])
      sec = 0

    base = datetime.datetime(year, month, day, hour, minute, sec)
    date_list = [base + datetime.timedelta(seconds=x) for x in range(0, t_length)]
    dates = np.array(date_list)
#     flghtnum= np.repeat(file_name, len(dates))
    # arr['timestamp'] = dates
    feature_dict['timestamp'] =  feature_dict['timestamp'] + (dates.T).tolist()
    
    # Filling all feature vector columns in the recarray
    for key in features_1:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = data[0:data.size:rate]
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = feature_dict[key] + (norm_v.T).tolist()[0]
    size_1hz = len(norm_v)
    for key in features_25:
        data =  raw_data[key]['data'][0][0]
        rate =  raw_data[key]['Rate'][0][0][0][0]
        dtype = raw_data[key]['data'][0][0].dtype
        # Add column to the recarray with proper dtype
        # arr = recfn.rec_append_fields(arr, key, arrs_1d, dtype)
        # Normalize vector to 1Hz by subsampling original vector
        norm_v = np.resize(data,(size_1hz,1))
        # Add normalized feature data vector to added column
        # arr[key] = norm_v.T
        feature_dict[key] = feature_dict[key] + (norm_v.T).tolist()[0]
        # print((norm_v.T).tolist()[0])
    return



In [6]:
def write_csv_output_s3(s3_key, feature_dict, bucket_name='iiaweb-s3-io-practice-bucket'):
    filename = s3_key.split(' ')[1].split('/')[2].split('.')[0]
    path = bucket_name+'/'+s3_key.split(' ')[1].rsplit('/',1)[0]+'/'+filename+'.gzip'
    df = pd.DataFrame.from_dict(feature_dict)
    s3 = s3fs.S3FileSystem(anon=False)
    with s3.open(path, 'wb') as f:
        gz = gzip.GzipFile(filename, mode='wb', compresslevel=9, fileobj=f)
        buf = io.TextIOWrapper(gz)
        df.to_csv(buf, index=False, encoding='UTF_8')
        gz.flush()
        gz.close()

In [7]:
exec_time ={}
for ind, key in enumerate(key_list):
    feature_dict = {}
    start = datetime.datetime.now()
    s3_key = key.key
    flght,set_num,date_s= s3_key.split('.')[0].split('/')
    tail= flght.split(' ')[1]
    if tail in ['657','658','659','660']:
        raw_mat = load_mat_file(s3_key)
        features_25, features_1, data_def= extract_features(raw_mat)
        feature_vector_array(raw_mat, features_25, features_1,date_s)
        feature_dictionary(feature_dict,raw_mat,features_25, features_1,date_s)
        write_csv_output_s3(s3_key, feature_dict)
        end = datetime.datetime.now()
        print("Time Taken to run",flght,set_num, s3_key, str(end-start))
        exec_time[s3_key]= str(end-start)
        del(raw_mat, features_25, features_1, data_def, feature_dict)

Time Taken to run Flight 657 1 Flight 657/1/657200103300305.mat 0:00:01.575517
Time Taken to run Flight 657 1 Flight 657/1/657200103300545.mat 0:00:02.140993
Time Taken to run Flight 657 1 Flight 657/1/657200103300747.mat 0:00:02.923593
Time Taken to run Flight 657 1 Flight 657/1/657200103301008.mat 0:00:01.545594
Time Taken to run Flight 657 1 Flight 657/1/657200103301138.mat 0:00:01.561585
Time Taken to run Flight 657 1 Flight 657/1/657200103301347.mat 0:00:03.240677
Time Taken to run Flight 657 1 Flight 657/1/657200103301612.mat 0:00:03.382311
Time Taken to run Flight 657 1 Flight 657/1/657200103301956.mat 0:00:00.687439
Time Taken to run Flight 657 1 Flight 657/1/657200103310002.mat 0:00:00.520481
Time Taken to run Flight 657 1 Flight 657/1/657200103310907.mat 0:00:03.972482
Time Taken to run Flight 657 1 Flight 657/1/657200103311147.mat 0:00:04.376799
Time Taken to run Flight 657 1 Flight 657/1/657200103311518.mat 0:00:01.570824
Time Taken to run Flight 657 1 Flight 657/1/65720010

KeyboardInterrupt: 

In [76]:
s3_key.split(' ')[1].split('/')[2].split('.')[0]

'652200101092046'

In [85]:
write_csv_output_s3(s3_key, feature_dict)